In this notebook we build a quantitative momentum investment strategy.

"Momentum investing" means investing in the stocks that have increased in price the most.

More precisely, we will

- select the 50 stocks (among the stocks composing S \& P 500) with the highest price momentum

- where the price momentum will be computed based on returns over various time periods and

- compute recommended trades to build an "equal-weight" position on these 50 stocks.

We start by retrieving (from wikipedia) the composition of the S \& P 500 and store it in a list

In [1]:
import pandas as pd

def get_sp_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_table = tables[0]
    sp500_tickers = sp500_table["Symbol"].tolist()
    sp500_tickers = [ticker.replace('.', '-') for ticker in sp500_tickers] # reformat the strings (e.g. BF.B -> BF-B) for yfinance
    return sp500_tickers

sp500_tickers = get_sp_tickers()

We now use yahoo finance api to get the returns of the S \& P 500 stocks over various time periods

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def get_1yr_returns(tickers):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=365)

    data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']
    data = data.dropna()

    latest_prices = data.iloc[-1]

    closest_date_1yr = min(data.index, key=lambda d: abs(d - (end_date - timedelta(days=365))))
    price_1yr = data.loc[closest_date_1yr]
    return_1yr = ((latest_prices - price_1yr) / price_1yr)

    df = pd.DataFrame({
        "Ticker": latest_prices.index,
        "Stock price": latest_prices.values,
        "1 year return %": 100 * return_1yr.values,
        "Strategy (nb shares to buy)": "N/A"
        })

    return df

sp_returns = get_1yr_returns(sp500_tickers)

YF.download() has changed argument auto_adjust default to True


In [3]:
sp_returns.head()

,Ticker,Stock price,1 year return %,Strategy (nb shares to buy)
0,A,102.709999,-18.075190,N/A
1,AAPL,198.149994,-12.731123,N/A
2,ABBV,175.050003,7.943779,N/A
3,ABNB,114.540001,-24.624904,N/A
4,ABT,126.879997,26.152354,N/A


A simple trading strategy would be to build an "equal-weight" porfolio of the top 50 "1 year return \%" stocks

In [4]:
sp_returns.sort_values("1 year return %", ascending = False, inplace = True)
sp_returns = sp_returns[:50]
sp_returns.reset_index(inplace = True)

In [5]:
sp_returns.head()

,index,Ticker,Stock price,1 year return %,Strategy (nb shares to buy)
0,374,PLTR,88.550003,219.675092,N/A
1,47,AXON,567.979980,87.774385,N/A
2,201,GEV,321.429993,84.476476,N/A
3,125,DASH,180.490005,65.526416,N/A
4,442,TPL,1236.099976,65.058150,N/A


Given a porfolio size, we can now compute the number of shares to buy to build an equal-weights position

In [6]:
import math
portfolio_size = 1000000
strategy = sp_returns.copy()
position_size = portfolio_size / len(sp_returns)
for i in range(len(sp_returns.index)):
    strategy.loc[i, "Strategy (nb shares to buy)"] = math.floor(position_size / strategy.loc[i, "Stock price"])

In [7]:
strategy.head()

,index,Ticker,Stock price,1 year return %,Strategy (nb shares to buy)
0,374,PLTR,88.550003,219.675092,225
1,47,AXON,567.979980,87.774385,35
2,201,GEV,321.429993,84.476476,62
3,125,DASH,180.490005,65.526416,110
4,442,TPL,1236.099976,65.058150,16


A better strategy would be to use various quantitative momentum, that is considering various time periods for the return. We can define a "high quality" momentum stock to be a stock showing a slow and steady outperformance over long periods of time (here, measured using price return over time periods, see also sharpe ratio, etc.).

Here, to identify high-quality momentum stock (HQM stock), we choose to select stocks with the highest percentiles of returns over the following periods of time: 1 month, 3 month, 6 month and 1 year.

Note: low-quality momentum often arise by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

In [8]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def get_multi_period_returns(tickers):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=365)

    data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']
    data = data.dropna()

    latest_prices = data.iloc[-1]

    def price_at_days_ago(days):
        target_date = end_date - timedelta(days=days)
        closest_date = min(data.index, key=lambda d: abs(d - target_date))
        return data.loc[closest_date]

    price_1mo = price_at_days_ago(30)
    price_3mo = price_at_days_ago(90)
    price_6mo = price_at_days_ago(180)
    price_1yr = price_at_days_ago(365)

    return_1mo = ((latest_prices - price_1mo) / price_1mo)
    return_3mo = ((latest_prices - price_3mo) / price_3mo)
    return_6mo = ((latest_prices - price_6mo) / price_6mo)
    return_1yr = ((latest_prices - price_1yr) / price_1yr)

    df = pd.DataFrame({
        "Ticker": latest_prices.index,
        "Stock price": latest_prices.values,
        "1 month return %": 100 * return_1mo.values,
        "1 month return percentile": "N/A",
        "3 month return %": 100 * return_3mo.values,
        "3 month return percentile": "N/A",
        "6 month return %": 100 * return_6mo.values,
        "6 month return percentile": "N/A",
        "1 year return %": 100 * return_1yr.values,
        "1 year return percentile": "N/A",
        "HQM score": "N/A",
        "Strategy (nb shares to buy)": 'N/A'
        })

    return df

sp_hqm = get_multi_period_returns(sp500_tickers)

In [9]:
sp_hqm.head()

,Ticker,Stock price,1 month return %,1 month return percentile,3 month return %,3 month return percentile,6 month return %,6 month return percentile,1 year return %,1 year return percentile,HQM score,Strategy (nb shares to buy)
0,A,102.709999,-15.061719,N/A,-28.238014,N/A,-25.872720,N/A,-18.075180,N/A,N/A,N/A
1,AAPL,198.149994,-7.185354,N/A,-14.965766,N/A,-14.321295,N/A,-12.731123,N/A,N/A,N/A
2,ABBV,175.050003,-17.339567,N/A,0.655511,N/A,-7.224219,N/A,7.943779,N/A,N/A,N/A
3,ABNB,114.540001,-6.771935,N/A,-10.235108,N/A,-15.249718,N/A,-24.624904,N/A,N/A,N/A
4,ABT,126.879997,0.134163,N/A,12.852439,N/A,8.254813,N/A,26.152363,N/A,N/A,N/A


Now, we compute momentum percentile scores for every stock using scipy.stats's percentileofscore method, computing the percentile rank of a score relative to a list of scores. For example if sp_hqm["6 month return percentile"] = 90 for Apple stock, this mean that 90 \% of the "6 month return %" are below Apple's "6 month return %" (hence apple has a really good 6 month return momentum percentile).

In [10]:
from scipy import stats

time_periods = ["1 month", "3 month", "6 month", "1 year"]
for row in sp_hqm.index:
    for time_period in time_periods:
        return_col = f"{time_period} return %"
        percentile_col = f"{time_period} return percentile"
        sp_hqm.loc[row, percentile_col] = stats.percentileofscore(sp_hqm[return_col], sp_hqm.loc[row, return_col])

In [11]:
sp_hqm.head()

,Ticker,Stock price,1 month return %,1 month return percentile,3 month return %,3 month return percentile,6 month return %,6 month return percentile,1 year return %,1 year return percentile,HQM score,Strategy (nb shares to buy)
0,A,102.709999,-15.061719,11.133201,-28.238014,5.964215,-25.872720,16.898608,-18.075180,20.477137,N/A,N/A
1,AAPL,198.149994,-7.185354,40.954274,-14.965766,31.212724,-14.321295,39.16501,-12.731123,25.646123,N/A,N/A
2,ABBV,175.050003,-17.339567,7.157058,0.655511,69.980119,-7.224219,57.455268,7.943779,62.624254,N/A,N/A
3,ABNB,114.540001,-6.771935,42.743539,-10.235108,43.737575,-15.249718,37.375746,-24.624904,13.916501,N/A,N/A
4,ABT,126.879997,0.134163,77.335984,12.852439,91.053678,8.254813,85.685885,26.152363,85.28827,N/A,N/A


We now compte a HQM score: here we consider the arithmetic mean of the percentiles

In [12]:
for row in sp_hqm.index:
    mean_percentiles = 0
    for time_period in time_periods:
        percentile_col = f"{time_period} return percentile"
        mean_percentiles += sp_hqm.loc[row, percentile_col]
    sp_hqm.loc[row, "HQM score"] = mean_percentiles / len(time_periods)

In [13]:
sp_hqm.head()

,Ticker,Stock price,1 month return %,1 month return percentile,3 month return %,3 month return percentile,6 month return %,6 month return percentile,1 year return %,1 year return percentile,HQM score,Strategy (nb shares to buy)
0,A,102.709999,-15.061719,11.133201,-28.238014,5.964215,-25.872720,16.898608,-18.075180,20.477137,13.61829,N/A
1,AAPL,198.149994,-7.185354,40.954274,-14.965766,31.212724,-14.321295,39.16501,-12.731123,25.646123,34.244533,N/A
2,ABBV,175.050003,-17.339567,7.157058,0.655511,69.980119,-7.224219,57.455268,7.943779,62.624254,49.304175,N/A
3,ABNB,114.540001,-6.771935,42.743539,-10.235108,43.737575,-15.249718,37.375746,-24.624904,13.916501,34.44334,N/A
4,ABT,126.879997,0.134163,77.335984,12.852439,91.053678,8.254813,85.685885,26.152363,85.28827,84.840954,N/A


Finally, we select our top 50 stocks according to our HQM score

In [14]:
sp_hqm.sort_values("HQM score", ascending = False, inplace = True)
sp_hqm = sp_hqm[:50]
sp_hqm.reset_index(inplace = True)

In [15]:
sp_hqm.head()

,index,Ticker,Stock price,1 month return %,1 month return percentile,3 month return %,3 month return percentile,6 month return %,6 month return percentile,1 year return %,1 year return percentile,HQM score,Strategy (nb shares to buy)
0,374,PLTR,88.550003,2.678578,86.878728,34.349868,99.602386,111.185315,100.0,219.675092,100.0,96.620278,N/A
1,472,VRSN,247.130005,3.683658,90.059642,18.244021,97.2167,32.112695,98.807157,40.542544,95.82505,95.477137,N/A
2,452,TTWO,212.070007,3.666227,89.860835,17.797039,96.819085,36.898846,99.602386,39.519742,95.228628,95.377734,N/A
3,375,PM,153.889999,2.217794,85.685885,31.044478,99.403579,29.847397,98.210736,54.726065,98.210736,95.377734,N/A
4,104,COR,284.329987,10.175525,98.210736,18.713165,97.813121,20.732263,96.222664,27.959341,87.475149,94.930417,N/A


and, given a porfolio size, compute the number of shares to buy to build an equal-weights position

In [16]:
import math
portfolio_size = 1000000
hqm_strategy = sp_hqm.copy()
position_size = portfolio_size / len(sp_hqm)
for i in range(len(sp_hqm.index)):
    hqm_strategy.loc[i, "Strategy (nb shares to buy)"] = math.floor(position_size / hqm_strategy.loc[i, "Stock price"])

In [17]:
hqm_strategy.head()

,index,Ticker,Stock price,1 month return %,1 month return percentile,3 month return %,3 month return percentile,6 month return %,6 month return percentile,1 year return %,1 year return percentile,HQM score,Strategy (nb shares to buy)
0,374,PLTR,88.550003,2.678578,86.878728,34.349868,99.602386,111.185315,100.0,219.675092,100.0,96.620278,225
1,472,VRSN,247.130005,3.683658,90.059642,18.244021,97.2167,32.112695,98.807157,40.542544,95.82505,95.477137,80
2,452,TTWO,212.070007,3.666227,89.860835,17.797039,96.819085,36.898846,99.602386,39.519742,95.228628,95.377734,94
3,375,PM,153.889999,2.217794,85.685885,31.044478,99.403579,29.847397,98.210736,54.726065,98.210736,95.377734,129
4,104,COR,284.329987,10.175525,98.210736,18.713165,97.813121,20.732263,96.222664,27.959341,87.475149,94.930417,70
